In [3]:
#Data____
#Authors: Rohan, Dakota, Chris, David
#Organize/clean the raw annotated data
import numpy as np
import pandas as pd;
import matplotlib.pyplot as plt
import tensorflow as tf

Thought process on working with the data:

We have the true predicted in the annotated data set as G, the annotators broke up the by selected text in the json.
So we decided to map selected text by policy ID further by that particular segmented ID with the class of G (10 class categorical)


In [4]:
#cleaning and organzing the data
filenames = ["/content/Annotations/1017_sci-news.com.csv", "/content/Annotations/1028_redorbit.com.csv","/content/Annotations/1050_honda.com.csv","/content/Annotations/105_amazon.com.csv","/content/Annotations/1106_allstate.com.csv",
             "/content/Annotations/1468_rockstargames.com.csv", "/content/Annotations/175_mlb.mlb.com.csv", "/content/Annotations/303_reddit.com.csv", "/content/Annotations/32_voxmedia.com.csv", "/content/Annotations/635_playstation.com.csv","/content/Annotations/856_sciencemag.org.csv",
             "/content/Annotations/988_solarviews.com.csv" ]

In [5]:
#create the dataFrames
totalData = 0
headers = ['AnnotationID', 'BATCHID', 'ANNOTATORID', 'POLICYID', 'SEGMENTID', 'categoryName','attributeValuePairs','PolicyURL','date']

# Assuming 'filenames' is a list of filenames containing your data
for i, filename in enumerate(filenames, 1):
    globals()[f'data_{i}'] = pd.read_csv(filename)
    globals()[f'data_{i}'] = pd.DataFrame(globals()[f'data_{i}'])
    globals()[f'data_{i}'] = globals()[f'data_{i}'].rename(columns=dict(zip(globals()[f'data_{i}'].columns, headers)))
    totalData += 1

#print(globals()[f'data_{1}'].head())
i = 1

# Creating a new dataframe containing all the data from all files.
newDF = pd.DataFrame()

while i <= totalData:
    print(f'The head of the {i} ')
    if isinstance(globals()[f'data_{i}'], pd.DataFrame):
        df = globals()[f'data_{i}']
        # Dropping specified columns
        columns_to_drop = ["AnnotationID", "BATCHID", "ANNOTATORID", "date"]
        df = df.drop(columns=columns_to_drop, axis=1)

        globals()[f'data_{i}'] = df
        #print(globals()[f'data_{i}'].head())

        #Adding all data to one DataFrame
        newDF = pd.concat([newDF, df], ignore_index=True)


    i += 1





The head of the 1 
The head of the 2 
The head of the 3 
The head of the 4 
The head of the 5 
The head of the 6 
The head of the 7 
The head of the 8 
The head of the 9 
The head of the 10 
The head of the 11 
The head of the 12 


So Now That we have data frames for 12 urls, we want to parse the json column to extract the line that generated the categoryName.

In [6]:
import json

i = 1
while i <= totalData:
    selected_texts = []
    df = globals()[f'data_{i}']

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        attribute_pairs = json.loads(row['attributeValuePairs'])

        # Iterate over the actual json array
        for key, value in attribute_pairs.items():
            selected_text = value.get('selectedText', '')
            selected_texts.append(selected_text)
        df["ActualText"] = selected_text
        globals()[f'data{i}'] = df
    #Add the extracted selected texts to a new column named "ActualText"
    #df['ActualText'] = selected_texts

    # Update the dataframe in globals()
    globals()[f'data{i}'] = df

    # Print the "ActualText" column for debugging
    print(globals()[f'data{i}'].head())

    i += 1  # Increment i to move to the next dataframe






   POLICYID  SEGMENTID                categoryName  \
0      3905          1  First Party Collection/Use   
1      3905          1  First Party Collection/Use   
2      3905          2              Data Retention   
3      3905          3  First Party Collection/Use   
4      3905          3  First Party Collection/Use   

                                 attributeValuePairs      PolicyURL  \
0  {"Collection Mode": {"selectedText": "nformati...  Not specified   
1  {"Collection Mode": {"selectedText": "nformati...  Not specified   
2  {"Personal Information Type": {"selectedText":...  Not specified   
3  {"Collection Mode": {"selectedText": "Not sele...  Not specified   
4  {"Collection Mode": {"selectedText": "Not sele...  Not specified   

              ActualText  
0  privacy@sci-news.com.  
1  privacy@sci-news.com.  
2  privacy@sci-news.com.  
3  privacy@sci-news.com.  
4  privacy@sci-news.com.  
   POLICYID  SEGMENTID                categoryName  \
0      3906          1          

In [7]:
#standardize the text data
for i in range(1, totalData + 1):
    if f'data_{i}' in globals() and isinstance(globals()[f'data_{i}'], pd.DataFrame):
        df = globals()[f'data_{i}']

        #remove grammar and convert to lowercase
        df['ActualText'] = df['ActualText'].apply(lambda x: x.lower().replace(',', '').replace('.', '').replace('!', '').replace('?', ''))
        globals()[f'data_{i}'] = df
    #print(globals()[f'data_{i}'].head())


### Condensing all data to one dataframe

In [8]:
# Creating a new dataframe containing all the data from all files.
newDF = pd.DataFrame()

i = 1

while i <= totalData:
    print(f'The head of the {i} ')
    if isinstance(globals()[f'data_{i}'], pd.DataFrame):
        df = globals()[f'data_{i}']
        globals()[f'data_{i}'] = df
        #print(globals()[f'data_{i}'].head())

        #Adding all data to one DataFrame
        newDF = pd.concat([newDF, df], ignore_index=True)


    i += 1

The head of the 1 
The head of the 2 
The head of the 3 
The head of the 4 
The head of the 5 
The head of the 6 
The head of the 7 
The head of the 8 
The head of the 9 
The head of the 10 
The head of the 11 
The head of the 12 


# MODEL

In [9]:
#!pip install transformers

## Get all the imports

In [10]:
import transformers
import torch
from transformers import AdamW
from transformers import BertTokenizer,\
BertForSequenceClassification

import torch.nn as nn

### Import Pretrained model

In [11]:
#import pretrained model

pretrained_model_name = 'bert-base-uncased'

### Load pre-learned model

In [12]:
#Loading the corpus of pre-learned. By loading this pre-trained model, we are leveraging the knowledge and insights gained from its pre-training.
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=10)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
newDF

,POLICYID,SEGMENTID,categoryName,attributeValuePairs,PolicyURL,ActualText
0,3905,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",Not specified,privacy@sci-newscom
1,3905,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati...",Not specified,privacy@sci-newscom
2,3905,2,Data Retention,"{""Personal Information Type"": {""selectedText"":...",Not specified,privacy@sci-newscom
3,3905,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele...",Not specified,privacy@sci-newscom
4,3905,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele...",Not specified,privacy@sci-newscom
...,...,...,...,...,...,...
2784,3835,0,Third Party Sharing/Collection,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...
2785,3835,0,Third Party Sharing/Collection,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...
2786,3835,0,Third Party Sharing/Collection,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...
2787,3835,0,Third Party Sharing/Collection,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...


 ### Label encoding the data

In [14]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'categoryName'.
print(newDF['categoryName'].unique())
newDF['categoryName']= label_encoder.fit_transform(newDF['categoryName'])

print(newDF['categoryName'].unique())

['First Party Collection/Use' 'Data Retention'
 'International and Specific Audiences' 'Other' 'User Choice/Control'
 'Third Party Sharing/Collection' 'User Access, Edit and Deletion'
 'Policy Change' 'Data Security' 'Do Not Track']
[3 0 4 5 9 7 8 6 1 2]


In [15]:
newDF

,POLICYID,SEGMENTID,categoryName,attributeValuePairs,PolicyURL,ActualText
0,3905,1,3,"{""Collection Mode"": {""selectedText"": ""nformati...",Not specified,privacy@sci-newscom
1,3905,1,3,"{""Collection Mode"": {""selectedText"": ""nformati...",Not specified,privacy@sci-newscom
2,3905,2,0,"{""Personal Information Type"": {""selectedText"":...",Not specified,privacy@sci-newscom
3,3905,3,3,"{""Collection Mode"": {""selectedText"": ""Not sele...",Not specified,privacy@sci-newscom
4,3905,3,3,"{""Collection Mode"": {""selectedText"": ""Not sele...",Not specified,privacy@sci-newscom
...,...,...,...,...,...,...
2784,3835,0,7,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...
2785,3835,0,7,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...
2786,3835,0,7,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...
2787,3835,0,7,"{""Third Party Entity"": {""selectedText"": ""third...",Not specified,collect certain anonymous information when you...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [16]:
input_texts = newDF['ActualText']
labels = newDF['categoryName']

input_ids = []
attention_masks = []

for text in input_texts:
	encoded_dict = tokenizer.encode_plus(
		text,
		# add_special_tokens=True, #Whether or not to add special tokens when encoding the sequences. This will use the underlying PretrainedTokenizerBase.build_inputs_with_special_tokens function, which defines which tokens are automatically added to the input ids. This is usefull if you want to add bos or eos tokens automatically.
		# max_length=128,
		padding='max_length',  # Maximum length (in tokens) to use for padding or truncation. Has no effect if tokenize is False. If not specified, the tokenizer’s max_length attribute will be used as a default.
		# truncation=True, #  Whether to truncate sequences at the maximum length. Has no effect if tokenize is False.
		return_tensors='pt'
	)

	input_ids.append(encoded_dict['input_ids'])
	attention_masks.append(encoded_dict['attention_mask'])



## Train Test Split

In [17]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import random

def shuffle(*args):
    # Check if all arguments have the same length
    lengths = [len(arg) for arg in args]
    if len(set(lengths)) != 1:
        raise ValueError("All arguments must have the same length.")

    # Generate a list of indices and shuffle them
    indices = list(range(lengths[0]))
    random.shuffle(indices)

    # Shuffle all input arguments based on the shuffled indices
    shuffled_args = []
    for arg in args:
        shuffled_args.append([arg[i] for i in indices])

    return tuple(shuffled_args)


# Assuming you have your input data and labels ready
#input_ids = [...]  # List of tokenized input ids
#attention_masks = [...]  # List of attention masks
#labels = [...]  # List of labels

# Shuffle the data and labels in the same order
input_ids_shuffled, attention_masks_shuffled, labels_shuffled = shuffle(input_ids, attention_masks, labels)

# Split the shuffled dataset into training and testing sets
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, train_labels, test_labels = \
    train_test_split(input_ids_shuffled, attention_masks_shuffled, labels_shuffled, test_size=0.2)
'''
# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(train_input_ids), torch.tensor(train_attention_masks), torch.tensor(train_labels))
test_dataset = TensorDataset(torch.tensor(test_input_ids), torch.tensor(test_attention_masks), torch.tensor(test_labels))

# Define batch size
batch_size = 16

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Now you can use train_dataloader and test_dataloader in your training and testing loops, respectively
'''


#input_ids = torch.cat(input_ids, dim=0)
#attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels)

train_input_ids = torch.cat(train_input_ids, dim=0)
test_input_ids = torch.cat(test_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

In [18]:
import torch
from transformers import BertForSequenceClassification

# Instantiate the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=10)

# Define the path to the saved model file
model_path = "best_model_Training42.pt"

# Load the model's state dictionary
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

#print(state_dict)

# Load the state dictionary into the model
model.load_state_dict(state_dict, strict=False)

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Fine Tuning the model

In [19]:
# Define the number of classes
num_classes = newDF['categoryName'].nunique()  # Change this to the desired number of classes

# Update the model architecture to handle the increased number of classes
# Assuming model is your neural network model
# Modify the output layer to have 'num_classes' units
model.fc = nn.Linear(in_features=1, out_features=num_classes)  # Update 'in_features' according to your model's architecture

print("LENGTH: ",len(train_input_ids))

batch_size = 16
epochs = 25
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define CrossEntropyLoss criterion
criterion = nn.CrossEntropyLoss()

# Fine-tuning loop with early stopping and model saving
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()  # The model is put into training mode using model.train().

# Define early stopping parameters
early_stopping_patience = 5
best_val_loss = float('inf')
best_model_path = "best_model.pt"
current_patience = 0

for epoch in range(epochs):  # epoch runs
    val_losses = []
    print(epoch)
    for i in range(0, train_input_ids.size(0), batch_size):  # batch runs
        batch_train_input_ids = train_input_ids[i:i+batch_size].to(device)
        batch_train_attention_masks = train_attention_masks[i:i+batch_size].to(device)
        batch_train_labels = train_labels[i:i+batch_size].to(device)

        optimizer.zero_grad()  # reset the gradient

        outputs = model(
            input_ids=batch_train_input_ids,
            attention_mask=batch_train_attention_masks
        )

        # Calculate the loss
        loss = criterion(outputs.logits, batch_train_labels)  # Accessing logits for loss calculation
        print(loss)

        val_losses.append(loss.item())

        loss.backward()
        optimizer.step()
    avg_val_loss = sum(val_losses) / len(val_losses)

    # Early stopping and model saving
    if avg_val_loss < best_val_loss:
        torch.save(model.state_dict(), best_model_path)
        best_val_loss = avg_val_loss
        current_patience = 0
    else:
        current_patience += 1
        if current_patience >= early_stopping_patience:
            print("Early stopping triggered.")
            break


LENGTH:  2231


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0


KeyboardInterrupt: 

## Evaluating the model

In [ ]:
import torch
from transformers import BertForSequenceClassification

# Instantiate the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=10)

# Define the path to the saved model file
model_path = "best_model.pt"

# Load the model's state dictionary
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

#print(state_dict)

# Load the state dictionary into the model
model.load_state_dict(state_dict, strict=False)

# If you want to use the model for inference, set it to evaluation mode
model.eval()


# old method that was not working model = TFBertForSequenceClassification.from_pretrained("best_model.pt", from_pt=True)

## Test Accuracy

In [ ]:
# Assuming you have imported the necessary libraries and defined your model as provided in the previous code snippet

# Tokenize test data
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

# Create a TensorDataset for test data
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Define batch size
batch_size = 16

# Create a DataLoader for test data
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Evaluate the model on the test set
model.eval()
total_samples = 0
total_correct = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_samples += labels.size(0)
        total_correct += (predicted_labels == labels).sum().item()

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)
